# L6: Multi-agent Collaboration for Financial Analysis


## SETUP


In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from crewai import Agent, Task, Crew
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

## TOOLS


In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

tool_search = SerperDevTool()
tool_scrape = ScrapeWebsiteTool()

In [4]:
tool_search, tool_scrape

(SerperDevTool(name='Search the internet', description="Search the internet(search_query: 'string') - A tool that can be used to semantic search a query from a txt's content.", args_schema=<class 'crewai_tools.tools.serper_dev_tool.serper_dev_tool.SerperDevToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x12fd69630>, search_url='https://google.serper.dev/search', n_results=None),
 ScrapeWebsiteTool(name='Read website content', description="Read website content(website_url: 'string') - A tool that can be used to read a website content.", args_schema=<class 'crewai_tools.tools.scrape_website_tool.scrape_website_tool.ScrapeWebsiteToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x12fd69630>, website_url=None, cookies=None, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,applicat

## AGENT


In [5]:
# agent to generate data insights
agent_data_analyst = Agent(
    role="Data Analyst",
    goal="""
Monitor and Analyze market data in real-time to identify trends and predict market movements
""",
    backstory="""
Specializing in financial markets, this agent uses statistical modeling and machine learning to provide crucial insights. With a knack for data, the Data Analyst Agent is the cornerstone for informing trading decisions.
""",
    verbose=True,
    allow_delegation=True,
    tools=[tool_scrape, tool_search],
)

In [6]:
# agent to develop trading strategies based on provided insights
agent_trading_strategy = Agent(
    role="Trading Strategy Developer",
    goal="""
Develop and test various trading strategies based on the insights from the Data Analyst Agent.
""",
    backstory="""
Equipped with a deep understanding of financial markets and quantitative analysis, this agent devises and refines trading strategies. It evaluates the performance of different approaches to determine the most profitable and risk-averse options.
""",
    verbose=True,
    allow_delegation=True,
    tools=[tool_scrape, tool_search],
)

In [7]:
# agent to perform execution strategy based on risk
agent_execution = Agent(
    role="Trade Advisor",
    goal="""
Evaluate and provide insights on the risks associated with potential trading activities.
""",
    backstory="""
This agent specializes in analyzing the timing, price, and logistical details of potential trades. By evaluating these factors, it provides well-founded suggestions for when and how trades should be executed to maximize efficiency and adherence to strategy.
""",
    allow_delegation=True,
    verbose=True,
    tools=[tool_scrape, tool_search],
)

In [8]:
agent_risk_management = Agent(
    role="Risk Advisor",
    goal="""
Evaluate and provide insights on the risks associated with potential trading activities.
""",
    backstory="""
Armed with a deep understanding of risk assessment models and market dynamics, this agent scrutinizes the potential risks of proposed trades. It offers a detailed analysis of risk exposure and suggests safeguards to ensure that trading activities align with the firm's risk tolerance.
""",
    verbose=True,
    allow_delegation=True,
    tools=[tool_scrape, tool_search],
)

## TASK


In [9]:
# task to analyze market data
task_data_analysis = Task(
    description=(
        """
Continuously monitor and analyze market data for the selected stock ({stock_selection}). Use statistical modeling and machine learning to identify trends and predict market movements.
"""
    ),
    expected_output=(
        """
Insights and alerts about significant market opportunities or threats for {stock_selection}
"""
    ),
    agent=agent_data_analyst,
)

In [10]:
# task to develop trading strategies
task_strategy_development = Task(
    description=(
        """
Develop and refine trading strategies based on the insights from the Data Analyst and user-defined risk tolerance ({risk_tolerance}). Consider trading preferences ({trading_strategy_preference}).
"""
    ),
    expected_output=(
        """
A set of potential trading strategies for {stock_selection} that align with the user's risk tolerance.
"""
    ),
    agent=agent_trading_strategy,
)

In [11]:
task_execution_planning = Task(
    description=(
        """
Analyze approved trading strategies to determine the best execution methods for {stock_selection}, considering current market conditions and optimal pricing.
"""
    ),
    expected_output=(
        """
Detailed execution plans suggesting how and when to execute trades for {stock_selection}.
"""
    ),
    agent=agent_execution,
)

In [12]:
task_risk_assessment = Task(
    description=(
        """
Evaluate the risks associated with the proposed trading strategies and execution plans for {stock_selection}. Provide a detailed analysis of potential risks and suggest mitigation strategies.
"""
    ),
    expected_output=(
        """
A comprehensive risk analysis report detailing potential risks and mitigation recommendations for {stock_selection}
"""
    ),
    agent=agent_risk_management,
)

## CREW


In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.7, api_key=OPENAI_API_KEY)

In [14]:
crew_financial_trading = Crew(
    agents=[
        agent_data_analyst,
        agent_trading_strategy,
        agent_execution,
        agent_risk_management,
    ],
    tasks=[
        task_data_analysis,
        task_strategy_development,
        task_execution_planning,
        task_risk_assessment,
    ],
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True,
)

In [15]:
inputs_financial_trading = {
    "stock_selection": "VOO",
    "initial_capital": "1000",
    "risk_tolerance": "Medium",
    "trading_strategy_preference": "Long Term",
    "new_impact_consideration": True,
}

In [16]:
result = crew_financial_trading.kickoff(inputs=inputs_financial_trading)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: 
Continuously monitor and analyze market data for the selected stock (VOO). Use statistical modeling and machine learning to identify trends and predict market movements.



> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for the selected stock (VOO) to the Data Analyst. They have the expertise in statistical modeling and machine learning required for this task.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Data Analyst",
"task": "Continuously monitor and analyze market data for the selected stock (VOO) using statistical modeling and machine learning to identify trends and predict market movements.",
"context": "The data analyst must stay updated on VOO market data, apply statistical modeling and machine learning techniques to analyze trends, and predict market movements to identify significant opportunities or threats."
}


> Ente

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [17]:
from IPython.display import Markdown

Markdown(result)

NameError: name 'result' is not defined